In [1]:
import random
import statistics
import math 
import numpy as np
from numpy import *
import csv
import os
import re
import matplotlib.pyplot as plt
import time

from io import StringIO

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve


In [2]:
print (os.getcwd())

/Users/wuyue/Desktop/analysis/code


# the path

In [3]:
path = "/Users/wuyue/Desktop/analysis/stock/"
files = os.listdir(path)
valid_file = []

for file in files:
    if file[-4:] == ".csv":
        valid_file.append(file)
        
print (len(valid_file))

5115


# define the fuctions will be used 

In [4]:
# get the sign of a number
def sgn(x):
    if x > 0:
        return (1)
    else:
        return (0)
    
  
def sgn_0(x):
    if x > 0:
        return (1)
    if x == 0:
        return (0)
    if x < 0:
        return (-1)
    
    
# the number of pos and neg in a list determine the general trend
def sgn_num(x):
    po = 0
    ne = 0
    for i in x:
        if i > 0:
            po += 1
        if i <= 0:
            ne += 1
    if po > ne:
        return (1)
    if po <= ne:
        return (-1)    

# the value of pos and neg sum determine the 
def sgn_total(x):
    po = 0
    ne = 0
    for i in x:
        if i > 0:
            po += i
        if i <= 0:
            ne += abs(i)
    if po > ne:
        return (1)
    if po <= ne:
        return (-1)
    
    
def sqrt_abs(x):
    if x > 0:
        return math.log(x, 10)
    if x == 0:
        return 0
    else:
        return -math.log(abs(x), 10)

In [5]:
def equal_split(l, m):
    n = int(math.ceil(len(l)/float(m)))
    return [l[i:i+n] for i in range(0, len(l), n)]

def chunks(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

def mix(l):
    res_sum = []
    for i in range(0, len(l[0])):
        res = []
        for lst in l:
            res.append(lst[i])
        res_sum.append(res)
    return res_sum

def cut(l, n):
    res = []
    for i in range(0, n):
        res.append(l[i::n])
    return res

def slic(l, n, m):
    res = []
    for i in range (1, n+1):
        res.append(l[60*i+1:60*i+m+1])
    return res

# building the training model

In [6]:
start = time.clock()

training_data_total = []
training_label_total = []

for file in valid_file:
    
    open_price = []
    high_price = []
    low_price = []
    close_price = []
    volume = []
    change = []
    typical_price = []
    middle_price = []
    
    
    
    
    file_name = path + file
    
    with open(file_name, "rU", encoding = "ISO-8859-1") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:            
            
            diff = float(row[4]) - float(row[1])
            tp = (float(row[2]) + float(row[3]) + float(row[4]))/3
            mp = (float(row[2]) + float(row[3]))/2
        
            # tp = typical price, mp = middle price
            
            row.append(float('%.3f' % diff))
            row.append(float('%.3f' % tp))
            row.append(float('%.3f' % mp))
        
        
            open_price.append(float(row[1]))
            open_price_part = open_price[-700:]
            open_price_list = open_price[-700:-100]
            open_price_list_split = chunks(open_price_list, 60)
            open_price_list_testing = open_price[-100:]
        
        
            high_price.append(float(row[2]))
            high_price_part = high_price[-700:]
            high_price_list = high_price[-700:-100]
            high_price_list_split = chunks(high_price_list, 60)
            high_price_list_testing = high_price[-100:]
            
            low_price.append(float(row[3]))
            low_price_part = low_price[-700:]
            low_price_list = low_price[-700:-100]
            low_price_list_split = chunks(low_price_list, 60)
            low_price_list_testing = low_price[-100:]
            
            close_price.append(float(row[4]))
            close_price_part = close_price[-700:]
            close_price_list = close_price[-700:-100]
            close_price_list_split = chunks(close_price_list, 60)
            close_price_list_testing = close_price[-100:]
            
            volume.append(float(row[5]))
            volume_part = volume[-700:]
            volume_list = volume[-700:-100]
            volume_list_split = chunks(volume_list, 60)
            volume_list_testing = volume[-100:]
            
            change.append(row[7])
            change_part = change[-700:]
            change_list = change[-700:-100]
            change_list_split = chunks(change_list, 60)
            change_list_testing = change[-100:]
            
            typical_price.append(row[8])
            typical_price_part = typical_price[-700:]
            typical_price_list = typical_price[-700:-100]
            typical_price_list_split = chunks(typical_price_list, 60)
            typical_price_list_testing = typical_price[-100:]
            
            middle_price.append(row[9])
            middle_price_part = middle_price[-700:]
            middle_price_list = middle_price[-700:-100]
            middle_price_list_split = chunks(middle_price_list, 60)
            middle_price_list_testing = middle_price[-100:]
    
    training_list = [open_price_list_split, high_price_list_split, low_price_list_split, close_price_list_split, \
                    volume_list_split, change_list_split, typical_price_list_split, middle_price_list_split]

    training_list_mixed = mix(training_list)

    
    
    
    stock_feature_dict = {}
    
    for item in training_list_mixed:
    
        pos_list = []
        abs_list = []
    
        for flt in item[5]:
            abs_list.append(abs(flt))
            if flt > 0:
                pos_list.append(flt)
    
        abs_sum = float('%.3f' % sum(abs_list))
        pos_sum = float('%.3f' % sum(pos_list))
        raw_rsi = (pos_sum/(1+abs_sum))*100
        RSI = float('%.3f' % raw_rsi)
        stock_feature_dict['RSI'] = RSI 
        
        
        
        
        indicator_list = [0.5] 
    
        money_flow_list = [vol*tp for vol, tp in zip(item[4], item[6])]
        total_money_flow = float('%.2f' % sum(money_flow_list))
    
        for i in range(len(item[6])-1):
            det = sgn(float('%.2f' % (item[6][i+1] - item[6][i])))
            indicator_list.append(det)
       
        positive_money_flow_list = [ind*mf for ind, mf in zip(indicator_list, money_flow_list)]
        positive_money_flow = float('%.2f' % sum(positive_money_flow_list))
        negative_money_flow = total_money_flow - positive_money_flow
        money_rate = (positive_money_flow/negative_money_flow)
    
        raw_mfi = 100-100/(1+money_rate)
        MFI = float('%.3f' % raw_mfi)
        stock_feature_dict['MFI'] = MFI   
    
    
        if len(item[3]) >= 3:
            raw_rsv = 100*(close_price_list[-1] - min(close_price_list))/(max(close_price_list) - min(close_price_list))
        RSV = float('%.3f' % raw_rsv)
        stock_feature_dict['RSV'] = RSV
    
    
    
        ax = item[3][-1] - item[3][-5]
        bx = item[3][-5]
        raw_roc = 100*ax/bx
        ROC = float('%.3f' % raw_roc)
        stock_feature_dict['ROC'] = ROC
    
    
        square_sum_5 = []
        TP_5 = mean(item[6][-5:])
        MA_5 = mean(item[3][-5:])
        for i in item[3][-5:]:
            square_sum_5.append((i-MA_5)**2)
        MD_5 = math.sqrt(mean(square_sum_5))
    
        raw_cci = (TP_5 - MA_5)/(MD_5*0.015)
        CCI = float('%.3f' % raw_cci)
        stock_feature_dict["CCI"] = CCI
    
    
        vol_para = list(map(lambda x: (2*x[0]-x[1]-x[2])/exp(x[1]-x[2]), zip(item[3], item[1], item[2])))
    
        va = []
        va_change_list = []
        va.append(item[4][0])
    
        for i in range(0, len(item[4])-1):
            va.append(va[i] + vol_para[i]*item[4][i+1])
    
        for i in range(0, len(va)-1):
            va_change_list.append(va[i+1] - va[i])
        va_change_rate = (va[-1] - va[0])/va[0]
    
        if abs(va_change_rate) > 0.1:
            VA = sgn(va_change_rate)
        else:
            VA = sgn_num(va_change_list)
        stock_feature_dict["VA"] = VA
    
    
    
    
        closing_change_list = []

        for i in range(0, len(item[3])-1):
            closing_change_list.append(item[3][i+1]-item[3][i])
   
        closing_price_list_pvt = item[3][1:]
        volume_list_pvt = volume_list[1:]
        pvt_list = list(map(lambda x: x[0]*x[1]/x[2], zip(closing_change_list, volume_list_pvt, closing_price_list_pvt)))
        raw_pvt = sqrt_abs(sum(pvt_list))
        PVT = float('%.3f' % raw_pvt)
        stock_feature_dict["PVT"] = PVT
    
    
    
    
        sign_closing_change_list = []
        for i in closing_change_list:
            sign_closing_change_list.append(sgn_0(i))
        obv_list = list(map(lambda x: x[0]*x[1], zip(sign_closing_change_list, item[4][1:])))
        raw_obv = sqrt_abs(sum(obv_list))
        OBV = float('%.3f' % raw_obv)
        stock_feature_dict["OBV"] = OBV
    
    
    
    
    
        exp_len = 49
        exp_starting = len(item[3]) - exp_len
        price_list_50 = [mean(item[3][:exp_starting])] + item[3][-exp_len:]
    
    
        const_50 = 2/51
        EMA_50 = []
        for i in range(1,len(price_list_50)):
            raw_ema_50 = const_50*price_list_50[i-1] + (1-const_50)*price_list_50[i]
            ema_50 = float('%.3f' % raw_ema_50)
            EMA_50.append(ema_50)
    
        const_25 = 2/26
        EMA_25 = []
        for i in range(26,len(price_list_50)):
            raw_ema_25 = const_25*price_list_50[i-1] + (1-const_25)*price_list_50[i]
            ema_25 = float('%.3f' % raw_ema_25)
            EMA_25.append(ema_25)
        
        EMA_50c = EMA_50[-24:]
        EMA_diff = list(map(lambda x: x[0]-x[1], zip(EMA_50c, EMA_25)))
        EMA_mean = np.mean(EMA_diff)*100
        EMA_diff_mean = float('%.5f' % np.mean(EMA_mean))
        stock_feature_dict['EMA_DIFF'] = EMA_diff_mean
        stock_feature_dict['EMA_TREND'] = sgn_num(EMA_diff)
        stock_feature_dict['EMA_OVERALL'] = sgn_total(EMA_diff)
    
    
    
    
    
        cr_pos = []
        cr_neg = []
    
        middle_price_list_c = item[7][:-1]
        closing_price_list_c = item[3][1:]
        cr_list = list(map(lambda x: x[0]-x[1], zip(middle_price_list_c, closing_price_list_c)))
        for i in cr_list:
            if i > 0:
                cr_pos.append(i)
            else:
                cr_neg.append(abs(i))
    
        cr_pos_sum = float('%.3f' % sum(cr_pos))
        cr_neg_sum = float('%.3f' % sum(cr_neg))
        raw_cr = 100*(cr_pos_sum/cr_neg_sum)
        CR = float('%.3f' % raw_cr)
        stock_feature_dict["CR"] = CR
    
    
    
        square_sum = []
        MA = mean(item[3])
        MB = mean(item[3][:-1])
    
        for i in item[3]:
            square_sum.append((i-MA)**2)
        MD = math.sqrt(mean(square_sum))
    
        raw_UP = MB+2*MD
        raw_DN = MB-2*MD
        UP = float('%.3f' % raw_UP)
        DN = float('%.3f' % raw_DN)
        stock_feature_dict["UP"] = UP
        stock_feature_dict["DN"] = DN
        
        
        if item[3][-1] > item[3][0]:
            stock_feature_dict["change_c"] = 1
        else:
            stock_feature_dict["change_c"] = -1
            
            
        stock_feature_list = list(stock_feature_dict.values())
        stock_feature_list = [-100 if math.isnan(x) else x for x in stock_feature_list]
        stock_feature_list = [100 if math.isinf(x) else x for x in stock_feature_list]
        
        
        training_data = stock_feature_list[:-1]
        training_data_total.append(training_data)
        
        training_label = stock_feature_list[-1]
        training_label_total.append(training_label)
        

print (len(training_data_total))


end = time.clock()
print ((end-start)/60, "min")


/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: 'U' mode is deprecated
/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:172: RuntimeWarning: divide by zero encountered in double_scalars
/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:156: RuntimeWarning: divide by zero encountered in double_scalars
/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:172: RuntimeWarning: invalid value encountered in double_scalars
/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:156: RuntimeWarning: invalid value encountered in double_scalars


51150
31.136723716666665 min


In [7]:
transformer = RobustScaler().fit(training_data_total)
data = transformer.transform(training_data_total)
print (len(data))

51150


# build the validation model 5 DAYS

In [8]:
start = time.clock()

validation_data_total = []
validation_label_total = []

for file in valid_file:
    
    open_price = []
    high_price = []
    low_price = []
    close_price = []
    volume = []
    change = []
    typical_price = []
    middle_price = []
    
    
    
    
    file_name = path + file
    
    with open(file_name, "rU", encoding = "ISO-8859-1") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:            
            
            diff = float(row[4]) - float(row[1])
            tp = (float(row[2]) + float(row[3]) + float(row[4]))/3
            mp = (float(row[2]) + float(row[3]))/2
        
            # tp = typical price, mp = middle price
            
            row.append(float('%.3f' % diff))
            row.append(float('%.3f' % tp))
            row.append(float('%.3f' % mp))
        
        
            open_price.append(float(row[1]))
            open_price_list = open_price[-700:]
            open_price_list_split_5 = slic(open_price_list, 9, 60)
            
            high_price.append(float(row[2]))
            high_price_list = high_price[-700:]
            high_price_list_split_5 = slic(high_price_list, 9, 60)
            
            low_price.append(float(row[3]))
            low_price_list = low_price[-700:]
            low_price_list_split_5 = slic(low_price_list, 9, 60)
            
            close_price.append(float(row[4]))
            close_price_list = close_price[-700:]
            close_price_list_split_5 = slic(close_price_list, 9, 60)
            
            volume.append(float(row[5]))
            volume_list = volume[-700:]
            volume_list_split_5 = slic(volume_list, 9, 60)
        
            change.append(row[7])
            change_list = change[-700:]
            change_list_split_5 = slic(change_list, 9, 60)
            
            typical_price.append(row[8])
            typical_price_list = typical_price[-700:]
            typical_price_list_split_5 = slic(typical_price_list, 9, 60)
            
            middle_price.append(row[9])
            middle_price_list = middle_price[-700:]
            middle_price_list_split_5 = slic(middle_price_list, 9, 60)
    
    validation_list = [open_price_list_split_5, high_price_list_split_5, low_price_list_split_5, close_price_list_split_5, \
                    volume_list_split_5, change_list_split_5, typical_price_list_split_5, middle_price_list_split_5]

    validation_list_mixed = mix(validation_list)

    
    
    
    stock_feature_dict = {}
    
    for item in validation_list_mixed:
    
        pos_list = []
        abs_list = []
    
        for flt in item[5]:
            abs_list.append(abs(flt))
            if flt > 0:
                pos_list.append(flt)
    
        abs_sum = float('%.3f' % sum(abs_list))
        pos_sum = float('%.3f' % sum(pos_list))
        raw_rsi = (pos_sum/(1+abs_sum))*100
        RSI = float('%.3f' % raw_rsi)
        stock_feature_dict['RSI'] = RSI 
        
        
        
        
        indicator_list = [0.5] 
    
        money_flow_list = [vol*tp for vol, tp in zip(item[4], item[6])]
        total_money_flow = float('%.2f' % sum(money_flow_list))
    
        for i in range(len(item[6])-1):
            det = sgn(float('%.2f' % (item[6][i+1] - item[6][i])))
            indicator_list.append(det)
       
        positive_money_flow_list = [ind*mf for ind, mf in zip(indicator_list, money_flow_list)]
        positive_money_flow = float('%.2f' % sum(positive_money_flow_list))
        negative_money_flow = total_money_flow - positive_money_flow
        money_rate = (positive_money_flow/(negative_money_flow+1))
    
        raw_mfi = 100-100/(1+money_rate)
        MFI = float('%.3f' % raw_mfi)
        stock_feature_dict['MFI'] = MFI   
    
    
        if len(item[3]) >= 3:
            raw_rsv = 100*(close_price_list[-1] - min(close_price_list))/(max(close_price_list) - min(close_price_list))
        RSV = float('%.3f' % raw_rsv)
        stock_feature_dict['RSV'] = RSV
    
    
    
        ax = item[3][-1] - item[3][-5]
        bx = item[3][-5]
        raw_roc = 100*ax/bx
        ROC = float('%.3f' % raw_roc)
        stock_feature_dict['ROC'] = ROC
    
    
        square_sum_5 = []
        TP_5 = mean(item[6][-5:])
        MA_5 = mean(item[3][-5:])
        for i in item[3][-5:]:
            square_sum_5.append((i-MA_5)**2)
        MD_5 = math.sqrt(mean(square_sum_5))
    
        raw_cci = (TP_5 - MA_5)/(MD_5*0.015)
        CCI = float('%.3f' % raw_cci)
        stock_feature_dict["CCI"] = CCI
    
    
        vol_para = list(map(lambda x: (2*x[0]-x[1]-x[2])/exp(x[1]-x[2]), zip(item[3], item[1], item[2])))
    
        va = []
        va_change_list = []
        va.append(item[4][0])
    
        for i in range(0, len(item[4])-1):
            va.append(va[i] + vol_para[i]*item[4][i+1])
    
        for i in range(0, len(va)-1):
            va_change_list.append(va[i+1] - va[i])
        va_change_rate = (va[-1] - va[0])/va[0]
    
        if abs(va_change_rate) > 0.1:
            VA = sgn(va_change_rate)
        else:
            VA = sgn_num(va_change_list)
        stock_feature_dict["VA"] = VA
    
    
    
    
        closing_change_list = []

        for i in range(0, len(item[3])-1):
            closing_change_list.append(item[3][i+1]-item[3][i])
   
        closing_price_list_pvt = item[3][1:]
        volume_list_pvt = volume_list[1:]
        pvt_list = list(map(lambda x: x[0]*x[1]/x[2], zip(closing_change_list, volume_list_pvt, closing_price_list_pvt)))
        raw_pvt = sqrt_abs(sum(pvt_list))
        PVT = float('%.3f' % raw_pvt)
        stock_feature_dict["PVT"] = PVT
    
    
    
    
        sign_closing_change_list = []
        for i in closing_change_list:
            sign_closing_change_list.append(sgn_0(i))
        obv_list = list(map(lambda x: x[0]*x[1], zip(sign_closing_change_list, item[4][1:])))
        raw_obv = sqrt_abs(sum(obv_list))
        OBV = float('%.3f' % raw_obv)
        stock_feature_dict["OBV"] = OBV
    
    
    
    
    
        exp_len = 4
        exp_starting = len(item[3]) - exp_len
        price_list_50 = [mean(item[3][:exp_starting])] + item[3][-exp_len:]
    
    
        const_50 = 2/6
        EMA_50 = []
        for i in range(1,len(price_list_50)):
            raw_ema_50 = const_50*price_list_50[i-1] + (1-const_50)*price_list_50[i]
            ema_50 = float('%.3f' % raw_ema_50)
            EMA_50.append(ema_50)
    
        const_25 = 2/3
        EMA_25 = []
        for i in range(3,len(price_list_50)):
            raw_ema_25 = const_25*price_list_50[i-1] + (1-const_25)*price_list_50[i]
            ema_25 = float('%.3f' % raw_ema_25)
            EMA_25.append(ema_25)
        
        EMA_50c = EMA_50[-2:]
        EMA_diff = list(map(lambda x: x[0]-x[1], zip(EMA_50c, EMA_25)))
        EMA_mean = np.mean(EMA_diff)*100
        EMA_diff_mean = float('%.5f' % np.mean(EMA_mean))
        stock_feature_dict['EMA_DIFF'] = EMA_diff_mean
        stock_feature_dict['EMA_TREND'] = sgn_num(EMA_diff)
        stock_feature_dict['EMA_OVERALL'] = sgn_total(EMA_diff)
    
    
    
    
    
        cr_pos = []
        cr_neg = []
    
        middle_price_list_c = item[7][:-1]
        closing_price_list_c = item[3][1:]
        cr_list = list(map(lambda x: x[0]-x[1], zip(middle_price_list_c, closing_price_list_c)))
        for i in cr_list:
            if i > 0:
                cr_pos.append(i)
            else:
                cr_neg.append(abs(i))
    
        cr_pos_sum = float('%.3f' % sum(cr_pos))
        cr_neg_sum = float('%.3f' % sum(cr_neg))
        raw_cr = 100*(cr_pos_sum/(1+cr_neg_sum))
        CR = float('%.3f' % raw_cr)
        stock_feature_dict["CR"] = CR
    
    
    
        square_sum = []
        MA = mean(item[3])
        MB = mean(item[3][:-1])
    
        for i in item[3]:
            square_sum.append((i-MA)**2)
        MD = math.sqrt(mean(square_sum))
    
        raw_UP = MB+2*MD
        raw_DN = MB-2*MD
        UP = float('%.3f' % raw_UP)
        DN = float('%.3f' % raw_DN)
        stock_feature_dict["UP"] = UP
        stock_feature_dict["DN"] = DN
        
        
        if item[3][-1] > item[3][0]:
            stock_feature_dict["change_c"] = 1
        else:
            stock_feature_dict["change_c"] = -1
            
            
        stock_feature_list = list(stock_feature_dict.values())
        stock_feature_list = [-100 if math.isnan(x) else x for x in stock_feature_list]
        stock_feature_list = [100 if math.isinf(x) else x for x in stock_feature_list]
        
        
        validation_data = stock_feature_list[:-1]
        validation_data_total.append(validation_data)
        
        validation_label = stock_feature_list[-1]
        validation_label_total.append(validation_label)
        

print (len(validation_data_total))


end = time.clock()
print ((end-start)/60, "min")


/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: 'U' mode is deprecated
/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:139: RuntimeWarning: divide by zero encountered in double_scalars
/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:155: RuntimeWarning: divide by zero encountered in double_scalars
/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:155: RuntimeWarning: invalid value encountered in double_scalars


46035
26.20882273333334 min


In [9]:
transformer_v = RobustScaler().fit(validation_data_total)
validation_data = transformer_v.transform(validation_data_total)

In [10]:
start = time.clock()

testing_data_total = []
testing_label_total = []
testing_list_total = []

count_term=1

for file in valid_file:
    
    open_price = []
    high_price = []
    low_price = []
    close_price = []
    volume = []
    change = []
    typical_price = []
    middle_price = []
    
    
    
    
    file_name = path + file
    
    
    with open(file_name, "rU", encoding = "ISO-8859-1") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:            
            
            diff = float(row[4]) - float(row[1])
            tp = (float(row[2]) + float(row[3]) + float(row[4]))/3
            mp = (float(row[2]) + float(row[3]))/2
        
            # tp = typical price, mp = middle price
            
            row.append(float('%.3f' % diff))
            row.append(float('%.3f' % tp))
            row.append(float('%.3f' % mp))
        
        
            open_price.append(float(row[1]))
            open_price_list_testing = open_price[-100:-40]
        
        
            high_price.append(float(row[2]))
            high_price_list_testing = high_price[-100:-40]
            
            low_price.append(float(row[3]))
            low_price_list_testing = low_price[-100:-40]
            
            close_price.append(float(row[4]))
            close_price_list_testing = close_price[-100:-40]
            
            volume.append(float(row[5]))
            volume_list_testing = volume[-100:-40]
            
            change.append(row[7])
            change_list_testing = change[-100:-40]
            
            typical_price.append(row[8])
            typical_price_list_testing = typical_price[-100:-40]
            
            middle_price.append(row[9])
            middle_price_list_testing = middle_price[-100:-40]
    
    testing_list = [open_price_list_testing, high_price_list_testing, low_price_list_testing, close_price_list_testing, \
                    volume_list_testing, change_list_testing, typical_price_list_testing, middle_price_list_testing]


    
    
    
    stock_feature_dict = {}
    
    
    
    pos_list = []
    abs_list = []
    
    for flt in testing_list[5]:
        abs_list.append(abs(flt))
        if flt > 0:
            pos_list.append(flt)
    
    abs_sum = float('%.3f' % sum(abs_list))
    pos_sum = float('%.3f' % sum(pos_list))
    raw_rsi = (pos_sum/(1+abs_sum))*100
    RSI = float('%.3f' % raw_rsi)
    stock_feature_dict['RSI'] = RSI 
        
        
        
        
    indicator_list = [0.5] 
    
    money_flow_list = [vol*tp for vol, tp in zip(testing_list[4], testing_list[6])]
    total_money_flow = float('%.2f' % sum(money_flow_list))
    
    for i in range(len(testing_list[6])-1):
        det = sgn(float('%.2f' % (testing_list[6][i+1] - testing_list[6][i])))
        indicator_list.append(det)
       
    positive_money_flow_list = [ind*mf for ind, mf in zip(indicator_list, money_flow_list)]
    positive_money_flow = float('%.2f' % sum(positive_money_flow_list))
    negative_money_flow = total_money_flow - positive_money_flow
    money_rate = (positive_money_flow/negative_money_flow)
    
    raw_mfi = 100-100/(1+money_rate)
    MFI = float('%.3f' % raw_mfi)
    stock_feature_dict['MFI'] = MFI   
    
    
    if len(testing_list[3]) >= 3:
        raw_rsv = 100*(close_price_list_testing[-1] - min(close_price_list_testing))/(max(close_price_list_testing) - min(close_price_list_testing))
    RSV = float('%.3f' % raw_rsv)
    stock_feature_dict['RSV'] = RSV
    
    
    
    ax = testing_list[3][-1] - testing_list[3][-5]
    bx = testing_list[3][-5]
    raw_roc = 100*ax/bx
    ROC = float('%.3f' % raw_roc)
    stock_feature_dict['ROC'] = ROC
    
    
    square_sum_5 = []
    TP_5 = mean(testing_list[6][-5:])
    MA_5 = mean(testing_list[3][-5:])
    for i in testing_list[3][-5:]:
        square_sum_5.append((i-MA_5)**2)
        MD_5 = math.sqrt(mean(square_sum_5))
    
    raw_cci = (TP_5 - MA_5)/(MD_5*0.015)
    CCI = float('%.3f' % raw_cci)
    stock_feature_dict["CCI"] = CCI
    
    
    vol_para = list(map(lambda x: (2*x[0]-x[1]-x[2])/exp(x[1]-x[2]), zip(testing_list[3], testing_list[1], testing_list[2])))
    
    va = []
    va_change_list = []
    va.append(testing_list[4][0])
    
    for i in range(0, len(testing_list[4])-1):
        va.append(va[i] + vol_para[i]*testing_list[4][i+1])
    
    for i in range(0, len(va)-1):
        va_change_list.append(va[i+1] - va[i])
    va_change_rate = (va[-1] - va[0])/va[0]
    
    if abs(va_change_rate) > 0.1:
        VA = sgn(va_change_rate)
    else:
        VA = sgn_num(va_change_list)
    stock_feature_dict["VA"] = VA
    
    
    
    
    closing_change_list = []

    for i in range(0, len(testing_list[3])-1):
        closing_change_list.append(testing_list[3][i+1]-testing_list[3][i])
   
    closing_price_list_pvt = testing_list[3][1:]
    volume_list_pvt = volume_list_testing[1:]
    pvt_list = list(map(lambda x: x[0]*x[1]/x[2], zip(closing_change_list, volume_list_pvt, closing_price_list_pvt)))
    raw_pvt = sqrt_abs(sum(pvt_list))
    PVT = float('%.3f' % raw_pvt)
    stock_feature_dict["PVT"] = PVT
    
    
    
    
    sign_closing_change_list = []
    for i in closing_change_list:
        sign_closing_change_list.append(sgn_0(i))
    obv_list = list(map(lambda x: x[0]*x[1], zip(sign_closing_change_list, testing_list[4][1:])))
    raw_obv = sqrt_abs(sum(obv_list))
    OBV = float('%.3f' % raw_obv)
    stock_feature_dict["OBV"] = OBV
    
    
    
    
    
    exp_len = 49
    exp_starting = len(testing_list[3]) - exp_len
    price_list_50 = [mean(testing_list[3][:exp_starting])] + testing_list[3][-exp_len:]
    
    
    const_50 = 2/51
    EMA_50 = []
    for i in range(1,len(price_list_50)):
        raw_ema_50 = const_50*price_list_50[i-1] + (1-const_50)*price_list_50[i]
        ema_50 = float('%.3f' % raw_ema_50)
        EMA_50.append(ema_50)
    
    const_25 = 2/26
    EMA_25 = []
    for i in range(26,len(price_list_50)):
        raw_ema_25 = const_25*price_list_50[i-1] + (1-const_25)*price_list_50[i]
        ema_25 = float('%.3f' % raw_ema_25)
        EMA_25.append(ema_25)
        
    EMA_50c = EMA_50[-24:]
    EMA_diff = list(map(lambda x: x[0]-x[1], zip(EMA_50c, EMA_25)))
    EMA_mean = np.mean(EMA_diff)*100
    EMA_diff_mean = float('%.5f' % np.mean(EMA_mean))
    stock_feature_dict['EMA_DIFF'] = EMA_diff_mean
    stock_feature_dict['EMA_TREND'] = sgn_num(EMA_diff)
    stock_feature_dict['EMA_OVERALL'] = sgn_total(EMA_diff)
    
    
    
    
    
    cr_pos = []
    cr_neg = []
    
    middle_price_list_c = testing_list[7][:-1]
    closing_price_list_c = testing_list[3][1:]
    cr_list = list(map(lambda x: x[0]-x[1], zip(middle_price_list_c, closing_price_list_c)))
    for i in cr_list:
        if i > 0:
            cr_pos.append(i)
        else:
            cr_neg.append(abs(i))
    
    cr_pos_sum = float('%.3f' % sum(cr_pos))
    cr_neg_sum = float('%.3f' % sum(cr_neg))
    raw_cr = 100*(cr_pos_sum/cr_neg_sum)
    CR = float('%.3f' % raw_cr)
    stock_feature_dict["CR"] = CR
    
    
    
    square_sum = []
    MA = mean(testing_list[3])
    MB = mean(testing_list[3][:-1])
    
    for i in testing_list[3]:
        square_sum.append((i-MA)**2)
    MD = math.sqrt(mean(square_sum))
    
    raw_UP = MB+2*MD
    raw_DN = MB-2*MD
    UP = float('%.3f' % raw_UP)
    DN = float('%.3f' % raw_DN)
    stock_feature_dict["UP"] = UP
    stock_feature_dict["DN"] = DN
        
        
        
    if testing_list[3][-1] > testing_list[3][0]:
        stock_feature_dict["change_c"] = 1
    else:
        stock_feature_dict["change_c"] = -1
            
            
    stock_feature_list = list(stock_feature_dict.values())
    stock_feature_list = [-100 if math.isnan(x) else x for x in stock_feature_list]
    stock_feature_list = [100 if math.isinf(x) else x for x in stock_feature_list]
        
        
    testing_data = stock_feature_list[:-1]
    testing_data_total.append(testing_data)
        
    testing_label = stock_feature_list[-1]
    testing_label_total.append(testing_label)
    
    
    
print (len(testing_list_total))


end = time.clock()
print ((end-start)/60, "min")


/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 'U' mode is deprecated
/Users/wuyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:135: RuntimeWarning: divide by zero encountered in double_scalars


0
2.0111024499999983 min


In [11]:
transformer_testing = RobustScaler().fit(testing_data_total)
data_testing = transformer_testing.transform(testing_data_total)

In [12]:

start = time.clock()

dt_val_acc = []
dt_testing_acc = []

log_val_acc = []
log_testing_acc = []

nb_val_acc = []
nb_testing_acc = []

svm_val_acc = []
svm_testing_acc = []

rf_val_acc = []
rf_testing_acc = []


training_data_split = cut(data, 9)
training_label_split = cut(training_label_total, 9)

validation_data_split = cut(validation_data, 9)
validation_label_split = cut(validation_label_total, 9)

# print (training_data_split[2])

for i in range (0, 9):
    training_sub_list = []
    training_sub_list.append(training_data_split[i])
    training_sub_list.append(training_label_split[i])
    
    validation_sub_list = []
    validation_sub_list.append(validation_data_split[i])
    validation_sub_list.append(validation_label_split[i])
    
    
    
    model1 = tree.DecisionTreeClassifier()
    dt_model = model1.fit(training_sub_list[0], training_sub_list[-1])
    dt_result = dt_model.predict(validation_sub_list[0])
    dt_testing_result = dt_model.predict(testing_data_total)
    
    print ("decision tree has accuracy on validation", accuracy_score(validation_sub_list[-1], dt_result))
    dt_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], dt_result), '.3f')))
    
    print ("decision tree has precision on validation", precision_score(validation_sub_list[-1], dt_result))
    print ("decision tree has recall on validation", recall_score(validation_sub_list[-1], dt_result))
    print (classification_report(validation_sub_list[-1], dt_result))
    print (confusion_matrix(validation_sub_list[-1], dt_result))
    print ("===========================")
    print ("decision tree has accuracy on testing", accuracy_score(testing_label_total, dt_testing_result))
    dt_testing_acc.append(float(format(accuracy_score(testing_label_total, dt_testing_result), '.3f')))
    
    print ("decision tree has precision on testing", precision_score(testing_label_total, dt_testing_result))
    print ("decision tree has recall on testing", recall_score(testing_label_total, dt_testing_result))
    print (classification_report(testing_label_total, dt_testing_result))
    print (confusion_matrix(testing_label_total, dt_testing_result))    
    print ("============================================================")
    
    
    
    model2 = LogisticRegression(random_state=0, solver='liblinear', multi_class='ovr')
    log_model = model2.fit(training_sub_list[0], training_sub_list[-1])
    log_result = log_model.predict(validation_sub_list[0])
    log_testing_result = log_model.predict(testing_data_total)
    
    print ("log reg has accuracy on validation", accuracy_score(validation_sub_list[-1], log_result))
    log_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], log_result), '.3f')))
    
    print ("log reg has precision on validation", precision_score(validation_sub_list[-1], log_result))
    print ("log reg has recall on validation", recall_score(validation_sub_list[-1], log_result))
    print (classification_report(validation_sub_list[-1], log_result))
    print (confusion_matrix(validation_sub_list[-1], log_result))
    print ("===========================")
    print ("log reg has accuracy on testing", accuracy_score(testing_label_total, log_testing_result))
    log_testing_acc.append(float(format(accuracy_score(testing_label_total, log_testing_result), '.3f')))
    
    print ("log reg has precision on testing", precision_score(testing_label_total, log_testing_result))
    print ("log reg has recall on testing", recall_score(testing_label_total, log_testing_result))
    print (classification_report(testing_label_total, log_testing_result))
    print (confusion_matrix(testing_label_total, log_testing_result)) 
    print ("============================================================")
   

    
    model3 = GaussianNB()
    nb_model = model3.fit(training_sub_list[0], training_sub_list[-1])
    nb_result = nb_model.predict(validation_sub_list[0])
    nb_testing_result = nb_model.predict(testing_data_total)
    
    print ("naive bayes has accuracy on validation", accuracy_score(validation_sub_list[-1], nb_result))
    nb_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], nb_result), '.3f')))
    
    print ("naive bayes has precision on validation", precision_score(validation_sub_list[-1], nb_result))
    print ("naive bayes has recall on validation", recall_score(validation_sub_list[-1], nb_result))
    print (classification_report(validation_sub_list[-1], nb_result))
    print (confusion_matrix(validation_sub_list[-1], nb_result))
    print ("===========================")
    print ("naive bayes has accuracy on testing", accuracy_score(testing_label_total, nb_testing_result))
    nb_testing_acc.append(float(format(accuracy_score(testing_label_total, nb_testing_result), '.3f')))
    
    print ("naive bayes has precision on testing", precision_score(testing_label_total, nb_testing_result))
    print ("naive bayes has recall on testing", recall_score(testing_label_total, nb_testing_result))
    print (classification_report(testing_label_total, nb_testing_result))
    print (confusion_matrix(testing_label_total, nb_testing_result)) 
    print ("============================================================")
    
    
    
    model4 = LinearSVC(random_state=0, tol=1e-5)
    svm_model = model4.fit(training_sub_list[0], training_sub_list[-1])
    svm_result = svm_model.predict(validation_sub_list[0])
    svm_testing_result = svm_model.predict(testing_data_total)
    
    print ("svm has accuracy on validation", accuracy_score(validation_sub_list[-1], svm_result))
    svm_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], svm_result), '.3f')))
    
    print ("svm has precision on validation", precision_score(validation_sub_list[-1], svm_result))
    print ("svm has recall on validation", recall_score(validation_sub_list[-1], svm_result))
    print (classification_report(validation_sub_list[-1], svm_result))
    print (confusion_matrix(validation_sub_list[-1], svm_result))
    print ("===========================")
    print ("svm has accuracy on testing", accuracy_score(testing_label_total, svm_testing_result))
    svm_testing_acc.append(float(format(accuracy_score(testing_label_total, svm_testing_result), '.3f')))
    
    print ("svm has precision on testing", precision_score(testing_label_total, svm_testing_result))
    print ("svm has recall on testing", recall_score(testing_label_total, svm_testing_result))
    print (classification_report(testing_label_total, svm_testing_result))
    print (confusion_matrix(testing_label_total, svm_testing_result)) 
    print ("============================================================")
    
    
    
    model5 = RandomForestClassifier(n_estimators=500, max_depth=2, random_state=0)
    rf_model = model5.fit(training_sub_list[0], training_sub_list[-1])
    rf_result = rf_model.predict(validation_sub_list[0])
    rf_testing_result = rf_model.predict(testing_data_total)
    
    print ("random forest has accuracy on validation", accuracy_score(validation_sub_list[-1], rf_result))
    rf_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], rf_result), '.3f')))
    
    print ("random forest has precision on validation", precision_score(validation_sub_list[-1], rf_result))
    print ("random forest has recall on validation", recall_score(validation_sub_list[-1], rf_result))
    print (classification_report(validation_sub_list[-1], rf_result))
    print (confusion_matrix(validation_sub_list[-1], rf_result))
    print ("===========================")
    print ("random forest has accuracy on testing", accuracy_score(testing_label_total, rf_testing_result))
    rf_testing_acc.append(float(format(accuracy_score(testing_label_total, rf_testing_result), '.3f')))
    
    print ("random forest has precision on testing", precision_score(testing_label_total, rf_testing_result))
    print ("random forest has recall on testing", recall_score(testing_label_total, rf_testing_result))
    print (classification_report(testing_label_total, rf_testing_result))
    print (confusion_matrix(testing_label_total, rf_testing_result)) 
    print ("============================================================")
    
    

    
end = time.clock()
print ((end-start), "sec")

decision tree has accuracy on validation 0.7980449657869013
decision tree has precision on validation 0.7144790257104194
decision tree has recall on validation 0.7983870967741935
              precision    recall  f1-score   support

          -1       0.86      0.80      0.83      3131
           1       0.71      0.80      0.75      1984

   micro avg       0.80      0.80      0.80      5115
   macro avg       0.79      0.80      0.79      5115
weighted avg       0.80      0.80      0.80      5115

[[2498  633]
 [ 400 1584]]
decision tree has accuracy on testing 0.4095796676441838
decision tree has precision on testing 0.6845238095238095
decision tree has recall on testing 0.03731343283582089
              precision    recall  f1-score   support

          -1       0.40      0.97      0.57      2033
           1       0.68      0.04      0.07      3082

   micro avg       0.41      0.41      0.41      5115
   macro avg       0.54      0.51      0.32      5115
weighted avg       0.57 

/Users/wuyue/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/wuyue/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/wuyue/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/wuyue/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/wuy

svm has accuracy on validation 0.8164222873900293
svm has precision on validation 0.6954732510288066
svm has recall on validation 0.9369959677419355
              precision    recall  f1-score   support

          -1       0.95      0.74      0.83      3131
           1       0.70      0.94      0.80      1984

   micro avg       0.82      0.82      0.82      5115
   macro avg       0.82      0.84      0.81      5115
weighted avg       0.85      0.82      0.82      5115

[[2317  814]
 [ 125 1859]]
svm has accuracy on testing 0.3976539589442815
svm has precision on testing 1.0
svm has recall on testing 0.0003244646333549643
              precision    recall  f1-score   support

          -1       0.40      1.00      0.57      2033
           1       1.00      0.00      0.00      3082

   micro avg       0.40      0.40      0.40      5115
   macro avg       0.70      0.50      0.28      5115
weighted avg       0.76      0.40      0.23      5115

[[2033    0]
 [3081    1]]
random forest h

/Users/wuyue/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


log reg has accuracy on validation 0.8422287390029326
log reg has precision on validation 0.7423187218353134
log reg has recall on validation 0.9105527638190954
              precision    recall  f1-score   support

          -1       0.93      0.80      0.86      3125
           1       0.74      0.91      0.82      1990

   micro avg       0.84      0.84      0.84      5115
   macro avg       0.84      0.85      0.84      5115
weighted avg       0.86      0.84      0.84      5115

[[2496  629]
 [ 178 1812]]
log reg has accuracy on testing 0.5530791788856305
log reg has precision on testing 0.9950248756218906
log reg has recall on testing 0.25957170668397145
              precision    recall  f1-score   support

          -1       0.47      1.00      0.64      2033
           1       1.00      0.26      0.41      3082

   micro avg       0.55      0.55      0.55      5115
   macro avg       0.73      0.63      0.53      5115
weighted avg       0.79      0.55      0.50      5115

[[202

...

log reg has accuracy on validation 0.7925708699902249
log reg has precision on validation 0.7988972721996518
log reg has recall on validation 0.8820890740147389
              precision    recall  f1-score   support

          -1       0.78      0.65      0.71      1994
           1       0.80      0.88      0.84      3121

   micro avg       0.79      0.79      0.79      5115
   macro avg       0.79      0.77      0.77      5115
weighted avg       0.79      0.79      0.79      5115

[[1301  693]
 [ 368 2753]]
log reg has accuracy on testing 0.6025415444770283
log reg has precision on testing 0.6025415444770283
log reg has recall on testing 1.0
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      2033
           1       0.60      1.00      0.75      3082

   micro avg       0.60      0.60      0.60      5115
   macro avg       0.30      0.50      0.38      5115
weighted avg       0.36      0.60      0.45      5115

[[   0 2033]
 [   0 3

/Users/wuyue/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


svm has accuracy on validation 0.8207233626588465
svm has precision on validation 0.7812659520163349
svm has recall on validation 0.9807753925024031
              precision    recall  f1-score   support

          -1       0.95      0.57      0.71      1994
           1       0.78      0.98      0.87      3121

   micro avg       0.82      0.82      0.82      5115
   macro avg       0.87      0.78      0.79      5115
weighted avg       0.85      0.82      0.81      5115

[[1137  857]
 [  60 3061]]
svm has accuracy on testing 0.8570869990224829
svm has precision on testing 0.9681401831939467
svm has recall on testing 0.7887735236859182
              precision    recall  f1-score   support

          -1       0.75      0.96      0.84      2033
           1       0.97      0.79      0.87      3082

   micro avg       0.86      0.86      0.86      5115
   macro avg       0.86      0.87      0.86      5115
weighted avg       0.88      0.86      0.86      5115

[[1953   80]
 [ 651 2431]]
ran

/Users/wuyue/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


svm has accuracy on validation 0.8504398826979472
svm has precision on validation 0.8156169848959818
svm has recall on validation 0.9604026845637584
              precision    recall  f1-score   support

          -1       0.93      0.70      0.80      2135
           1       0.82      0.96      0.88      2980

   micro avg       0.85      0.85      0.85      5115
   macro avg       0.87      0.83      0.84      5115
weighted avg       0.86      0.85      0.85      5115

[[1488  647]
 [ 118 2862]]
svm has accuracy on testing 0.8694037145650049
svm has precision on testing 0.961038961038961
svm has recall on testing 0.8163530175210902
              precision    recall  f1-score   support

          -1       0.77      0.95      0.85      2033
           1       0.96      0.82      0.88      3082

   micro avg       0.87      0.87      0.87      5115
   macro avg       0.87      0.88      0.87      5115
weighted avg       0.89      0.87      0.87      5115

[[1931  102]
 [ 566 2516]]
rand

In [13]:
print (dt_val_acc, end = "\n")
print (dt_testing_acc, end = " ")

[0.798, 0.809, 0.86, 0.859, 0.816, 0.806, 0.827, 0.834, 0.82]
[0.41, 0.781, 0.357, 0.733, 0.458, 0.463, 0.214, 0.552, 0.602] 

In [14]:
print (log_val_acc, end = "\n")
print (log_testing_acc, end = " ")

[0.815, 0.842, 0.871, 0.593, 0.854, 0.844, 0.86, 0.793, 0.822]
[0.397, 0.553, 0.397, 0.572, 0.397, 0.397, 0.398, 0.603, 0.602] 

In [15]:
print (nb_val_acc, end = "\n")
print (nb_testing_acc, end = " ")

[0.408, 0.42, 0.423, 0.806, 0.702, 0.698, 0.811, 0.622, 0.633]
[0.397, 0.398, 0.397, 0.603, 0.397, 0.397, 0.397, 0.335, 0.311] 

In [16]:
print (svm_val_acc, end = "\n")
print (svm_testing_acc, end = " ")

[0.816, 0.829, 0.87, 0.906, 0.852, 0.845, 0.859, 0.821, 0.85]
[0.398, 0.397, 0.397, 0.397, 0.398, 0.398, 0.398, 0.857, 0.869] 

In [17]:
print (rf_val_acc, end = "\n")
print (rf_testing_acc, end = " ")

[0.835, 0.846, 0.888, 0.904, 0.865, 0.867, 0.86, 0.876, 0.885]
[0.798, 0.801, 0.803, 0.799, 0.799, 0.799, 0.798, 0.801, 0.802] 